In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# File paths
base_path = "C:\\Users\\Lucas\\Desktop\\Optimization\\Project1\\"
files = {
    "child_care": base_path + "child_care_regulated.csv",
    "income": base_path + "avg_individual_income.csv",
    "locations": base_path + "potential_locations.csv",
    "employment": base_path + "employment_rate.csv",
    "population": base_path + "population.csv"
}

# Load datasets
child_care = pd.read_csv(files['child_care'])
income = pd.read_csv(files['income'])
locations = pd.read_csv(files['locations'])
employment = pd.read_csv(files['employment'])
population = pd.read_csv(files['population'])

# Preprocess datasets and then merge necessary datasets by zip code
data = population.merge(income, left_on='zipcode', right_on='ZIP code')
data = data.merge(employment, on='zipcode')
data = data.merge(child_care, left_on='zipcode', right_on='zip_code', how='left')

# Fill NaN values for facilities without data
data.fillna({'total_capacity': 0}, inplace=True)

# Define constants and fairness constraints
BUDGET = 100_000_000
FAIRNESS_CONSTRAINT = 0.1
WEIGHT_UNDER_5 = 2
WEIGHT_ALL_CHILDREN = 1

# Calculate target coverage per zip code
child_population = data['-5'] + data['5-9'] + data['10-14']
under_5_population = data['-5']
data['target_coverage'] = (WEIGHT_UNDER_5 * under_5_population + WEIGHT_ALL_CHILDREN * child_population) / (WEIGHT_UNDER_5 + WEIGHT_ALL_CHILDREN)

In [ ]:
# Method 1: LP approach using scipy.optimize.linprog
cost_per_slot = 1000  # Assumed average cost per slot
num_zipcodes = len(data)

# Objective: Minimize the number of slots added (which correlates with cost)
c = np.ones(num_zipcodes) * cost_per_slot

# Constraints
A_ub = []
b_ub = []

# Fairness constraints: Ensure that the difference in coverage between areas is no more than FAIRNESS_CONSTRAINT
for i in range(num_zipcodes):
    for j in range(i + 1, num_zipcodes):
        row = np.zeros(num_zipcodes)
        row[i] = 1 / data.loc[i, 'target_coverage']
        row[j] = -1 / data.loc[j, 'target_coverage']
        A_ub.append(row)
        b_ub.append(FAIRNESS_CONSTRAINT)

A_ub = np.array(A_ub)
b_ub = np.array(b_ub)

# Budget constraint
A_eq = np.ones((1, num_zipcodes))
b_eq = [BUDGET / cost_per_slot]

# Bounds for the number of slots to add
bounds = [(0, None) for _ in range(num_zipcodes)]

# Solve the LP problem
lp_solution = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

if lp_solution.success:
    data['slots_to_add_lp'] = lp_solution.x
    print("Linear Programming optimization successful.")
    print(data[['zipcode', 'slots_to_add_lp']])
else:
    print("Linear Programming optimization failed:", lp_solution.message)

In [ ]:
# Method 2: IP approach using PuLP
# Create the problem
ip_problem = LpProblem("Child_Care_Fairness", LpMaximize)

# Decision variables
slots_to_add = {i: LpVariable(f'slots_to_add_{i}', lowBound=0, cat='Integer') for i in range(num_zipcodes)}

# Objective function: Maximize total slots added within the budget
ip_problem += lpSum(slots_to_add[i] for i in range(num_zipcodes))

# Fairness constraints
for i in range(num_zipcodes):
    for j in range(i + 1, num_zipcodes):
        ip_problem += (slots_to_add[i] / data.loc[i, 'target_coverage'] - slots_to_add[j] / data.loc[j, 'target_coverage']) <= FAIRNESS_CONSTRAINT

# Budget constraint
ip_problem += lpSum(slots_to_add[i] * cost_per_slot for i in range(num_zipcodes)) <= BUDGET

# Solve the IP problem
ip_problem.solve()

# Results
if ip_problem.status == 1:
    data['slots_to_add_ip'] = [slots_to_add[i].varValue for i in range(num_zipcodes)]
    print("Integer Programming optimization successful.")
    print(data[['zipcode', 'slots_to_add_ip']])
else:
    print("Integer Programming optimization failed.")

In [ ]:
# Method 3: 
# Step 1: Preprocess data
# Combine relevant datasets, such as population, child care capacity, etc., into a single DataFrame
# For example, merge population data with child care data by zip code:
merged_data = pd.merge(child_care_regulated, population, left_on='zip_code', right_on='zipcode')

# Merge with income, employment rate, and potential locations
merged_data = pd.merge(merged_data, avg_individual_income, left_on='zip_code', right_on='ZIP code')
merged_data = pd.merge(merged_data, employment_rate, on='zipcode')
merged_data = pd.merge(merged_data, potential_locations, on='zipcode')

# Step 2: Define the optimization problem
# Create a PuLP problem instance
prob = LpProblem("Maximize_Social_Coverage_Index", LpMaximize)

# Step 3: Define decision variables
# For each region, define variables for new slots and expanded slots
new_slots = {zip_code: LpVariable(f"new_slots_{zip_code}", lowBound=0, cat='Continuous') for zip_code in merged_data['zip_code']}
expanded_slots = {zip_code: LpVariable(f"expanded_slots_{zip_code}", lowBound=0, cat='Continuous') for zip_code in merged_data['zip_code']}

# Step 4: Define the objective function
# Maximize the social coverage index with 2:1 weight for younger children
objective = lpSum([2 * new_slots[zip_code] for zip_code in merged_data['zip_code']] + [expanded_slots[zip_code] for zip_code in merged_data['zip_code']])

prob += objective

# Step 5: Define constraints
# Example: Ensure child care deserts are eliminated by ensuring sufficient slots
for zip_code in merged_data['zip_code']:
    prob += new_slots[zip_code] + expanded_slots[zip_code] >= 0.5 * merged_data.loc[merged_data['zip_code'] == zip_code, 'Total'].values[0]

# Fairness constraint: Ensure the difference in coverage between regions does not exceed 0.1
for i in range(len(merged_data)):
    for j in range(i + 1, len(merged_data)):
        prob += abs((new_slots[merged_data['zip_code'].iloc[i]] + expanded_slots[merged_data['zip_code'].iloc[i]]) - 
                   (new_slots[merged_data['zip_code'].iloc[j]] + expanded_slots[merged_data['zip_code'].iloc[j]])) <= 0.1

# Budget constraint
total_budget = 100_000_000
# Add your budget constraints here based on the costs of new and expanded slots
# (e.g., sum of costs should not exceed the budget)

# Step 6: Solve the problem
prob.solve()

# Step 7: Print the results
for zip_code in merged_data['zip_code']:
    print(f"{zip_code}: New slots: {new_slots[zip_code].varValue}, Expanded slots: {expanded_slots[zip_code].varValue}")